<a href="https://colab.research.google.com/github/Lazyeee/Terraria-Modding-Tools/blob/main/_Lazysterrariamoddingtool_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install necessary things
import os
import subprocess
from IPython.display import display, HTML

# Update and install system dependencies
print("Updating packages and installing dependencies...")
subprocess.run(['apt-get', 'update'], check=True)
subprocess.run(['apt-get', 'install', '-y',
               'dotnet-sdk-8.0',
               'software-properties-common',
               'lib32gcc-s1',
               'build-essential',
               'zlib1g-dev'], check=True)

# Install ILSpy
print("\nInstalling ILSpy...")
subprocess.run(['dotnet', 'tool', 'install', '--global', 'ilspycmd'], capture_output=True, text=True)
# Add .dotnet/tools to PATH if needed
dotnet_tools_path = os.path.expanduser("~/.dotnet/tools")
if dotnet_tools_path not in os.environ["PATH"]:
    os.environ["PATH"] = f"{dotnet_tools_path}:{os.environ['PATH']}"


code = r"""
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <filesystem>
#include <zlib.h>

namespace fs = std::filesystem;

// Utility to read a 7-bit encoded integer (used for string lengths in .tmod)
int read7BitEncodedInt(std::ifstream& file) {
    int result = 0;
    int shift = 0;
    uint8_t byte;
    do {
        file.read(reinterpret_cast<char*>(&byte), 1);
        result |= (byte & 0x7F) << shift;
        shift += 7;
    } while (byte & 0x80);
    return result;
}

// Read a string with 7-bit encoded length
std::string readString(std::ifstream& file) {
    int length = read7BitEncodedInt(file);
    std::vector<char> buffer(length);
    file.read(buffer.data(), length);
    return std::string(buffer.begin(), buffer.end());
}

// Decompress DEFLATE-compressed data using zlib
std::vector<char> decompressData(const std::vector<char>& compressed, int uncompressedSize) {
    std::vector<char> decompressed(uncompressedSize);
    z_stream stream = {};
    stream.zalloc = Z_NULL;
    stream.zfree = Z_NULL;
    stream.opaque = Z_NULL;
    stream.avail_in = compressed.size();
    stream.next_in = reinterpret_cast<Bytef*>(const_cast<char*>(compressed.data()));
    stream.avail_out = uncompressedSize;
    stream.next_out = reinterpret_cast<Bytef*>(decompressed.data());

    inflateInit2(&stream, -15); // -15 for raw DEFLATE (no zlib header)
    int ret = inflate(&stream, Z_FINISH);
    inflateEnd(&stream);

    if (ret != Z_STREAM_END) {
        std::cerr << "Decompression failed: " << ret << std::endl;
        return {};
    }
    return decompressed;
}

void unpackTmod(const std::string& tmodPath, const std::string& outputDir) {
    std::ifstream file(tmodPath, std::ios::binary);
    if (!file.is_open()) {
        std::cerr << "Failed to open .tmod file: " << tmodPath << std::endl;
        return;
    }

    // Verify header
    char header[4];
    file.read(header, 4);
    if (std::string(header, 4) != "TMOD") {
        std::cerr << "Invalid .tmod file: header mismatch" << std::endl;
        return;
    }

    // Read version
    std::string version = readString(file);
    std::cout << "tModLoader Version: " << version << std::endl;

    // Skip hash (20 bytes) and signature (256 bytes)
    file.seekg(20 + 256, std::ios::cur);

    // Read file data length
    uint32_t fileDataLength;
    file.read(reinterpret_cast<char*>(&fileDataLength), 4);
    std::cout << "File data length: " << fileDataLength << " bytes" << std::endl;

    // Read mod name and version
    std::string modName = readString(file);
    std::string modVersion = readString(file);
    std::cout << "Mod Name: " << modName << ", Version: " << modVersion << std::endl;

    // Read file count
    int32_t fileCount;
    file.read(reinterpret_cast<char*>(&fileCount), 4);
    std::cout << "File Count: " << fileCount << std::endl;

    // Create output directory
    fs::create_directories(outputDir);

    // Read file entries
    struct FileEntry {
        std::string name;
        int32_t uncompressedLength;
        int32_t compressedLength;
        std::vector<char> compressedData;
    };
    std::vector<FileEntry> entries;
    for (int i = 0; i < fileCount; ++i) {
        FileEntry entry;
        entry.name = readString(file);
        file.read(reinterpret_cast<char*>(&entry.uncompressedLength), 4);
        file.read(reinterpret_cast<char*>(&entry.compressedLength), 4);
        entries.push_back(entry);
    }

    // Read and decompress file data
    for (auto& entry : entries) {
        std::vector<char> compressed(entry.compressedLength);
        file.read(compressed.data(), entry.compressedLength);
        entry.compressedData = compressed;

        std::vector<char> decompressed;
        if (entry.compressedLength != entry.uncompressedLength) {
            decompressed = decompressData(compressed, entry.uncompressedLength);
            if (decompressed.empty()) {
                std::cerr << "Failed to decompress: " << entry.name << std::endl;
                continue;
            }
        } else {
            decompressed = compressed; // No compression
        }

        // Write to file
        std::string outputPath = outputDir + "/" + entry.name;
        fs::create_directories(fs::path(outputPath).parent_path());
        std::ofstream outFile(outputPath, std::ios::binary);
        outFile.write(decompressed.data(), decompressed.size());
        outFile.close();
        std::cout << "Extracted: " << entry.name << std::endl;
    }

    file.close();
    std::cout << "\nUnpacking complete!\n";

    // Custom footer
    std::cout << "----------------------------------------\n";
    std::cout << "Custom Footer:\n";
    std::cout << "© 2025 xAI, Inc.\n";
    std::cout << "Developed with love by Grok 3\n";
    std::cout << "For more info, visit: https://x.ai\n";
    std::cout << "----------------------------------------\n";
}

int main(int argc, char* argv[]) {
    if (argc != 3) {
        std::cerr << "Usage: " << argv[0] << " <tmod_file> <output_directory>" << std::endl;
        return 1;
    }

    std::string tmodPath = argv[1];
    std::string outputDir = argv[2];
    unpackTmod(tmodPath, outputDir);
    return 0;
}
"""
with open("main.cpp", "w") as f:
    f.write(code.strip('\n'))
print("\nCompiling tmod_unpacker...")
compile_result = subprocess.run(
    ['g++', 'main.cpp', '-o', 'tmod_unpacker', '-lz', '-O3'],
    capture_output=True,
    text=True
)

if compile_result.returncode != 0:
    print("Compilation failed!")
    print(compile_result.stderr)
else:
    # Make executable and move to PATH
    subprocess.run(['chmod', '+x', 'tmod_unpacker'], check=True)
    subprocess.run(['mv', 'tmod_unpacker', '/usr/local/bin/'], check=True)

Updating packages and installing dependencies...

Installing ILSpy...

Compiling tmod_unpacker...


In [ ]:
#@title Install SteamCMD
!sudo add-apt-repository -y multiverse > /dev/null 2>&1
!sudo dpkg --add-architecture i386 > /dev/null 2>&1
!sudo apt-get update -qq
!sudo apt-get install -y -qq lib32gcc1 lib32stdc++6 libc6-i386 libcurl4-gnutls-dev:i386 > /dev/null 2>&1

!mkdir -p /content/steamcmd > /dev/null 2>&1
!cd /content/steamcmd && curl -sqL "https://steamcdn-a.akamaihd.net/client/installer/steamcmd_linux.tar.gz" | tar zxvf - > /dev/null 2>&1
!chmod +x /content/steamcmd/steamcmd.sh > /dev/null 2>&1

!/content/steamcmd/steamcmd.sh +quit > /dev/null 2>&1
print("SteamCMD installed successfully!")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
SteamCMD installed successfully!


In [ ]:
#@title Upload dll and get decompiled source code

import os
import shutil
from google.colab import files

uploaded = files.upload()
uploaded_filename = next(iter(uploaded))

output_folder = os.path.join('temp', uploaded_filename)
os.makedirs(output_folder, exist_ok=True)

!ilspycmd -p -o "{output_folder}" "{uploaded_filename}"

zip_filename = f"{uploaded_filename}_decompiled.zip"
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', 'temp', uploaded_filename)

files.download(zip_filename)
!rm -rf temp
!rm -rf "{uploaded_filename}"

Saving Terrandomizer.dll to Terrandomizer.dll
=

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Unpack tmod
import os
import shutil
from google.colab import files
from IPython.display import display, HTML

def process_tmod():
    # Upload file
    uploaded = files.upload()
    tmod_files = [f for f in uploaded.keys() if f.endswith('.tmod')]

    if not tmod_files:
        display(HTML('<h3 style="color:red">No .tmod file found! Please upload a valid tmod file.</h3>'))
        return

    if len(tmod_files) > 1:
        display(HTML('<h3 style="color:red">Multiple files detected! Please upload only one .tmod file.</h3>'))
        return

    tmod_filename = tmod_files[0]
    base_name = os.path.splitext(tmod_filename)[0]
    output_dir = f"temp/{base_name}"
    zip_name = f"{base_name}_unpacked.zip"

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    try:
        # Unpack the tmod file
        print(f"Unpacking {tmod_filename}...")
        result = subprocess.run(
            ['tmod_unpacker', tmod_filename, output_dir],
            check=True,
            capture_output=True,
            text=True
        )

        # Check if unpacking succeeded
        if result.returncode == 0:
            print(f"Successfully unpacked to: {output_dir}")

            # Create zip archive
            print(f"Creating {zip_name}...")
            shutil.make_archive(base_name, 'zip', output_dir)

            # Download the zip
            print("Preparing download...")
            files.download(f"{base_name}.zip")

            display(HTML(f'<h3 style="color:green">Success! Download will start automatically.</h3>'))
        else:
            display(HTML(f'<h3 style="color:red">Error unpacking file:</h3><pre>{result.stderr}</pre>'))

    except Exception as e:
        display(HTML(f'<h3 style="color:red">Error occurred:</h3><pre>{str(e)}</pre>'))
    finally:
        # Cleanup temporary files
        shutil.rmtree(output_dir, ignore_errors=True)
        if os.path.exists(tmod_filename):
            os.remove(tmod_filename)

# Run the processor
process_tmod()

Saving Terrandomizer.tmod to Terrandomizer.tmod
Unpacking Terrandomizer.tmod...
Successfully unpacked to: temp/Terrandomizer
Creating Terrandomizer_unpacked.zip...
Preparing download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Workshop Downloader
import os, subprocess, shutil, json, re
from google.colab import files

# === USER INPUTS ===
app_id      = 105600          #@param {type:"integer"}
workshop_id = "3515638241"    #@param {type:"string"}
# ===================

def sanitize_filename(name: str) -> str:
    return re.sub(r'[<>:"/\\|?*\x00-\x1f]', '_', name).strip()[:150]

if not workshop_id.isdigit():
    raise ValueError("Workshop ID must be numeric.")

# --- 1) Download via SteamCMD ---
install_dir = f"/content/workshop_mods/{app_id}/{workshop_id}"
os.makedirs(install_dir, exist_ok=True)

cmd = [
    "/content/steamcmd/steamcmd.sh",
    "+force_install_dir", install_dir,
    "+login", "anonymous",
    "+workshop_download_item", str(app_id), workshop_id,
    "+quit"
]
print("🔧 Fetching Workshop item...")
proc = subprocess.run(cmd, capture_output=True, text=True)
print(proc.stdout); print(proc.stderr)
if proc.returncode != 0:
    raise RuntimeError("SteamCMD failed—check logs above.")

# --- 2) Locate the actual mod folder ---
mod_dir = None
# common path
common = f"/content/steamapps/workshop/content/{app_id}/{workshop_id}"
if os.path.isdir(common) and os.listdir(common):
    mod_dir = common
# fallback to install_dir
elif os.path.isdir(install_dir) and os.listdir(install_dir):
    mod_dir = install_dir
# as a last resort, scan for any folder named workshop_id
else:
    for root, dirs, files_ in os.walk("/content"):
        if workshop_id in root and os.listdir(root):
            mod_dir = root
            break

if not mod_dir:
    raise FileNotFoundError("Could not locate the downloaded mod folder.")
print(f"✅ Mod folder found at: {mod_dir}")

# --- 3) Find and parse pack.json anywhere under mod_dir ---
pack_json_path = None
for root, dirs, files_ in os.walk(mod_dir):
    if "pack.json" in files_:
        pack_json_path = os.path.join(root, "pack.json")
        break

mod_name = None
if pack_json_path:
    try:
        with open(pack_json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        candidate = data.get("Name") or data.get("name")
        if isinstance(candidate, str) and candidate.strip():
            mod_name = sanitize_filename(candidate)
            print(f"🔍 pack.json found at {pack_json_path}; using Name = {mod_name}")
    except Exception as e:
        print(f"⚠️ Error reading pack.json: {e}")

# --- 4) Fallback to directory name or ID ---
if not mod_name:
    # use last segment of mod_dir
    mod_name = sanitize_filename(os.path.basename(mod_dir)) or workshop_id
    print(f"🔍 pack.json missing or empty; falling back to folder name = {mod_name}")

# --- 5) Zip and download ---
zip_name = f"mod_{mod_name}.zip"
shutil.make_archive(zip_name[:-4], 'zip', mod_dir)
print(f"📦 Created archive: {zip_name}")
files.download(zip_name)

Starting download...
Creating zip archive...
Preparing download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>